# Pathway enrichment

* Using `msigdbr` package in R to download GSEA pathway gene sets

In [1]:
#set wd
getwd()
setwd('/scratch/user/s4436039/scdata/Pathway-sets')
getwd()

[1] "/scratch/user/s4436039/Nikita-project"

[1] "/scratch/user/s4436039/scdata/Pathway-sets"

In [3]:
library(msigdbr)

In [4]:
msigdbr_collections()

gs_cat,gs_subcat,num_genesets
<chr>,<chr>,<int>
C1,,299
C2,CGP,3384
C2,CP,29
C2,CP:BIOCARTA,292
C2,CP:KEGG,186
C2,CP:PID,196
C2,CP:REACTOME,1615
C2,CP:WIKIPATHWAYS,664
C3,MIR:MIRDB,2377


In [5]:
msigdbr_species()

species_name,species_common_name
<chr>,<chr>
Anolis carolinensis,"Carolina anole, green anole"
Bos taurus,"bovine, cattle, cow, dairy cow, domestic cattle, domestic cow, ox, oxen"
Caenorhabditis elegans,NA
Canis lupus familiaris,"dog, dogs"
Danio rerio,"leopard danio, zebra danio, zebra fish, zebrafish"
Drosophila melanogaster,fruit fly
Equus caballus,"domestic horse, equine, horse"
Felis catus,"cat, cats, domestic cat"
Gallus gallus,"bantam, chicken, chickens, Gallus domesticus"


## Pathways of interest: 
* Cat: H --> Hallmark gene sets (well-defined biological states or processes)
* Cat: C2 --> curated gene sets --> KEGG_MEDICUS

In [ ]:
#get KEGG human gene sets
KEGG_sets = msigdbr(species = "human", category = "C2")
head(h_gene_sets)